In [18]:
import librosa

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random

from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch
import torchmetrics
import os

import pickle

In [35]:
import warnings
warnings.filterwarnings('ignore')

In [78]:
mixup_df = pd.read_csv('./output_mixed_audio_with_bandstop/mixed_labels.csv')
mixup_df

,filename,label
0,mixed_0_1.ogg,"[1.0, 0.0]"
1,mixed_2_3.ogg,"[1.0, 0.0]"
2,mixed_4_5.ogg,"[0.4999033690360951, 0.5000966309639049]"
3,mixed_6_7.ogg,"[0.0, 1.0]"
4,mixed_8_9.ogg,"[0.0, 1.0]"
...,...,...
27714,mixed_55428_55429.ogg,"[0.031195525974676782, 0.9688044740253232]"
27715,mixed_55430_55431.ogg,"[0.0, 1.0]"
27716,mixed_55432_55433.ogg,"[0.07274193603161008, 0.92725806396839]"
27717,mixed_55434_55435.ogg,"[0.0, 1.0]"


In [60]:
mixup_mfcc = np.load('./mfcc/mixup_with_bandstop_train_mfcc.npy')
mixup_labels = np.load('./mfcc/mixup_with_bandstop_train_labels.npy')
mixup_val_mfcc = np.load('./mfcc/mixup_with_bandstop_val_mfcc.npy')
mixup_val_labels = np.load('./mfcc/mixup_with_bandstop_val_labels.npy')

In [63]:
class Config:
    SR = 32000
    N_MFCC = 20
    # Dataset
    ROOT_FOLDER = './'
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 50
    LR = 3e-4
    # Others
    SEED = 42

CONFIG = Config()

In [64]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED) # Seed 고정

In [65]:
class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

In [66]:
train_dataset = CustomDataset(mixup_mfcc, mixup_labels)
val_dataset = CustomDataset(mixup_val_mfcc, mixup_val_labels)

In [67]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

In [68]:
class MLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=128, output_dim=CONFIG.N_CLASSES):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

In [69]:
from sklearn.metrics import mean_squared_error


def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)

            probs = model(features)

            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

        _val_loss = np.mean(val_loss)

        all_labels = np.vstack(all_labels)
        all_probs = np.vstack(all_probs)

        # Calculate MSE and MAE
        mse_score = mean_squared_error(all_labels, all_probs)
    
    return _val_loss, mse_score



def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)

    best_val_score = float('inf')
    best_model = None

    for epoch in range(1, CONFIG.N_EPOCHS + 1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)

            optimizer.zero_grad()

            output = model(features)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, mse_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val MSE : [{mse_score:.5f}]]')

        if best_val_score > mse_score:
            best_val_score = mse_score
            best_model = model

    return best_model


In [70]:
device = device = torch.device("mps")

In [71]:
model = MLP()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CONFIG.LR)

infer_model = train(model, optimizer, train_loader, val_loader, device)

100%|██████████| 58/58 [00:00<00:00, 373.36it/s]


Epoch [1], Train Loss : [0.63864] Val Loss : [0.55583] Val MSE : [0.13172]]


100%|██████████| 58/58 [00:00<00:00, 827.00it/s]


Epoch [2], Train Loss : [0.53717] Val Loss : [0.55270] Val MSE : [0.12916]]


100%|██████████| 58/58 [00:00<00:00, 849.16it/s]


...

Epoch [48], Train Loss : [0.38044] Val Loss : [0.39628] Val MSE : [0.06737]]


100%|██████████| 58/58 [00:00<00:00, 896.03it/s]


Epoch [49], Train Loss : [0.38293] Val Loss : [0.39382] Val MSE : [0.06712]]


100%|██████████| 58/58 [00:00<00:00, 878.68it/s]

Epoch [50], Train Loss : [0.37834] Val Loss : [0.40106] Val MSE : [0.06750]]


In [72]:
def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features

In [73]:
test_mfcc = np.load('./mfcc/test_mfcc.npy')

test = pd.read_csv('./test.csv')
#test_mfcc = get_mfcc_feature(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)
#np.save('./mfcc/test_mfcc.npy', test_mfcc)

In [74]:
test_mfcc.shape

(50000, 20)

In [75]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            print(features)
            features = features.float().to(device)

            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [76]:
preds = inference(infer_model, test_loader, device)

 40%|███▉      | 207/521 [00:00<00:00, 1053.83it/s]

tensor([[-182.1974,  101.5643,  -19.3814,  ...,   -7.8198,   -6.4962,
           -5.7868],
        [-258.6904,  124.4828,  -10.8553,  ...,   -8.4540,   -7.1144,
           -5.5447],
        [-246.2839,  124.1053,  -14.6470,  ...,   -7.6716,   -1.5682,
           -3.0644],
        ...,
        [ -82.5985,   62.3140,   20.8452,  ...,    6.4835,   -6.5670,
            5.1510],
        [-203.4839,  106.2778,  -32.0374,  ...,    1.6235,   -1.8640,
            2.2522],
        [-217.2787,  129.8681,  -28.8649,  ...,   -9.8326,   -5.4955,
           -6.4186]])
tensor([[-132.6274,  151.7619,   -0.5873,  ...,    4.0435,    1.4240,
            5.2067],
        [-262.1068,   90.5862,  -44.4091,  ...,   18.5748,    4.4177,
            7.7497],
        [-180.0639,   73.7920,  -47.0841,  ...,   -9.6733,   -1.5734,
           -5.1682],
        ...,
        [-163.6234,  112.0763,   -8.4852,  ...,   -1.2879,    1.7295,
            2.0518],
        [-172.9889,  105.9831,   -8.1232,  ...,   -3.0881,   -9

 60%|██████    | 313/521 [00:00<00:00, 858.89it/s] 

tensor([[-275.7150,  128.5501,  -25.3169,  ...,   -9.7081,   -5.0938,
           -4.4096],
        [-103.3735,  143.9074,  -25.6135,  ...,    4.8703,   -3.1160,
           -1.8800],
        [-267.1035,  108.0646,  -24.8068,  ...,  -10.9131,   -3.5071,
           -4.3342],
        ...,
        [-513.8236,   17.0007,   -8.5985,  ...,    4.9441,    1.5639,
           -6.3698],
        [-220.3637,   96.0707,    7.5140,  ...,   -1.4235,   -2.9628,
            7.9631],
        [-298.5253,  115.2568,  -29.0495,  ...,   -2.9116,   -0.6831,
           -1.1764]])
tensor([[-1.2993e+02,  1.8444e+02, -4.6617e+01,  ..., -5.3782e+00,
         -2.9088e+00, -1.7229e-01],
        [-2.4989e+02,  1.0760e+02, -1.1882e+01,  ..., -9.0470e+00,
         -6.1507e+00, -7.7286e+00],
        [-1.0537e+02,  1.2564e+02,  3.4306e+01,  ...,  2.9421e+00,
         -7.1946e+00,  9.4624e-01],
        ...,
        [-3.6596e+02,  9.0203e+01, -9.6935e+00,  ..., -8.7134e+00,
         -6.9670e+00, -4.5123e+00],
        [-2.130

 83%|████████▎ | 433/521 [00:00<00:00, 975.15it/s]

tensor([[-3.7733e+02,  8.6600e+01, -1.6368e+01,  ..., -4.7053e+00,
          6.5054e-01, -5.3898e+00],
        [-1.0760e+02,  7.5196e+01,  3.2022e-01,  ..., -6.1585e-01,
         -1.0611e+01,  3.2958e+00],
        [-3.1668e+02,  2.4284e+01, -1.5485e+01,  ...,  4.9581e+00,
         -9.1745e+00,  2.0093e+00],
        ...,
        [-8.1169e+01,  6.6524e+01, -3.0386e+01,  ...,  6.8316e+00,
         -3.6353e+00,  1.5622e+00],
        [-4.9097e+02,  8.1676e+01, -2.3736e+01,  ..., -6.6699e+00,
         -4.7245e+00, -1.6922e+00],
        [-2.3539e+02,  1.1289e+02, -1.9506e+01,  ..., -6.3600e+00,
         -3.3667e+00, -5.0380e+00]])
tensor([[-1.0105e+02,  7.9874e+01, -3.8978e+01,  ..., -8.9084e+00,
         -9.9660e+00, -1.2130e+00],
        [-3.8403e+02,  8.4484e+01,  3.3481e+01,  ...,  4.5547e+00,
         -2.6555e+00,  8.3888e-01],
        [-2.3386e+02,  1.4952e+02, -2.2015e+01,  ..., -3.8087e-01,
          3.4105e+00, -4.3296e-01],
        ...,
        [-1.8157e+02,  9.4710e+01,  4.6051e+00

100%|██████████| 521/521 [00:00<00:00, 845.09it/s]

tensor([[-2.1219e+02,  1.4281e+02, -4.0217e+01,  ..., -4.7249e+00,
         -4.1832e+00, -6.5490e+00],
        [-3.4487e+02,  8.2610e+01, -1.7817e+01,  ..., -4.1693e+00,
         -4.5574e+00, -4.3456e+00],
        [-1.8066e+02,  1.7836e+02, -1.6500e+01,  ...,  1.2171e+00,
          4.3992e+00,  3.8245e+00],
        ...,
        [-2.5573e+02,  1.2004e+02, -2.3382e+01,  ..., -1.2940e+01,
         -4.2788e+00, -6.6304e+00],
        [-2.5921e+02,  1.3586e+02, -2.5291e+01,  ...,  1.1760e-01,
          2.3707e+00,  3.5248e+00],
        [-2.4193e+02,  7.7141e+01, -4.8922e+00,  ..., -3.6992e+00,
         -3.8001e+00, -2.9602e+00]])


In [77]:
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

,id,fake,real
0,TEST_00000,1.0,4.107405e-09
1,TEST_00001,1.0,1.030222e-10
2,TEST_00002,1.0,7.222934e-10
3,TEST_00003,1.0,2.208570e-11
4,TEST_00004,1.0,2.566532e-13


In [56]:
submit.to_csv('./mixup_with_baseDataset_MLP_n_mfcc=20.csv', index=False)